In [42]:
import qumulo
import os
import io
import glob
import re
import time
from datetime import datetime
import dateutil.parser as date_parser
from qumulo.rest_client import RestClient

# Client

In [25]:
API_HOSTNAME = os.environ['API_HOSTNAME'] if 'API_HOSTNAME' in os.environ else '10.12.54.21'
API_USER = os.environ['API_USER'] if 'API_USER' in os.environ else 'admin'
API_PASSWORD = os.environ['API_PASSWORD'] if 'API_PASSWORD' in os.environ else 'P@ssw0rd'

In [26]:
from qumulo.rest_client import RestClient

rc = RestClient('10.12.54.21',8000)
rc.login('admin','P@ssw0rd')

rc.fs.read_fs_stats()

{'block_size_bytes': 4096,
 'total_size_bytes': '77491947438080',
 'free_size_bytes': '77489912471552',
 'snapshot_size_bytes': '0'}

In [40]:
all_nodes = rc.cluster.list_nodes()
for node in all_nodes:
    print(node)

{'id': 1, 'node_status': 'online', 'node_name': 'qumulo-HD-1', 'uuid': 'cabeaae8-2d0c-4201-8620-572049b97605', 'label': '88:e9:a4:20:26:ca', 'model_number': 'HPE34T_DL325_G10Plus', 'serial_number': 'SGH139YVBG', 'mac_address': '88:e9:a4:20:26:ca'}
{'id': 2, 'node_status': 'online', 'node_name': 'qumulo-HD-2', 'uuid': '77bb97ef-f5ef-4084-a416-741907e68afe', 'label': '88:e9:a4:20:66:ea', 'model_number': 'HPE34T_DL325_G10Plus', 'serial_number': 'SGH139YVBT', 'mac_address': '88:e9:a4:20:66:ea'}
{'id': 3, 'node_status': 'online', 'node_name': 'qumulo-HD-3', 'uuid': '58dd5459-c105-4ef0-9737-5a20410763d9', 'label': '88:e9:a4:20:66:72', 'model_number': 'HPE34T_DL325_G10Plus', 'serial_number': 'SGH139YVC5', 'mac_address': '88:e9:a4:20:66:72'}
{'id': 4, 'node_status': 'online', 'node_name': 'qumulo-HD-4', 'uuid': 'acedd1b5-1151-4608-9f66-68f7826d4422', 'label': '88:e9:a4:20:36:22', 'model_number': 'HPE34T_DL325_G10Plus', 'serial_number': 'SGH139YVB3', 'mac_address': '88:e9:a4:20:36:22'}


# Qumulo Python SDK Class

In [28]:
qumulo_lib_path = os.path.dirname(qumulo.__file__) + '/rest'

total_matches = 0
for f in glob.glob(qumulo_lib_path + '/*.py'):
    file_name = os.path.basename(f)
    if file_name == '__init__.py':
        continue
    print("")
    print("-"*80)
    print("Area: %s" % (file_name, ))
    c = open(f, 'r').read()
    rx_str = '@request.request[ \r\n]+def ([^(]+)\([ \r\n]*conninfo,[ \r\n]*credentials([^\)]*)(.*?)(return|yield)'
    ms = re.findall(rx_str, c, re.S|re.M)
    for m in ms:
        total_matches += 1
        func_name = m[0]
        
        # get arguments
        args = []
        arg_ms = m[1].split(',')
        for arg_m in arg_ms:
            if arg_m.strip() != "":
                args.append(re.sub('=.*', '', arg_m.strip()))

        # method
        method = "GET"
        method_m = re.search('method[ ]*=[ ]*"([A-Z]+)', m[2])
        if method_m is not None:
            method = method_m.group(1)

        # uri, currently more work for fs methods
        uri = "/"
        uri_m = re.search('uri[ ]*=.*?"([^"]+)', m[2])
        if uri_m is not None:
            uri = uri_m.group(1)
        uri_m = re.search('uri[ ]*=.*?\'([^\']+)', m[2])
        if uri_m is not None:
            uri = uri_m.group(1)

        print("   rc.%s.%s(%s)" % (file_name.replace('.py', ''), 
                                   func_name, 
                                   ', '.join(args[:4]) + (' ...' if len(args)>4 else '')))


--------------------------------------------------------------------------------
Area: checksumming.py

--------------------------------------------------------------------------------
Area: network.py

--------------------------------------------------------------------------------
Area: node_state.py

--------------------------------------------------------------------------------
Area: metrics.py

--------------------------------------------------------------------------------
Area: upgrade_v2.py

--------------------------------------------------------------------------------
Area: s3.py

--------------------------------------------------------------------------------
Area: version.py

--------------------------------------------------------------------------------
Area: ftp.py

--------------------------------------------------------------------------------
Area: time_config.py

--------------------------------------------------------------------------------
Area: object_replicat

# NFS 

In [50]:
rc.nfs.get_nfs_config()

{'v4_enabled': False,
 'krb5_enabled': True,
 'krb5p_enabled': True,
 'krb5i_enabled': True,
 'auth_sys_enabled': True}

# S3 Bucket

In [51]:
rc.s3.create_bucket(name='test1',create_path='/home/')

BucketDescription(name='test1', creation_time='2023-06-15T12:58:10.714743379Z', path='/csi/volumes1/test1', anonymous_access_enabled=False)

In [52]:
rc.s3.list_buckets()

BucketDescriptionList(buckets=[BucketDescription(name='test1', creation_time='2023-06-15T12:58:10.714743379Z', path='/csi/volumes1/test1', anonymous_access_enabled=False)])

In [43]:
rc.s3.delete_bucket(name='test1',delete_root_dir=False)

In [61]:
rc.s3.list_buckets()

BucketDescriptionList(buckets=[BucketDescription(name='test1', creation_time='2023-06-15T12:58:10.714743379Z', path='/csi/volumes1/test1', anonymous_access_enabled=False)])

In [ ]:
rc.s3.list_uploads()